# Instalacion de dependencias

In [2]:
#pip install spacy

In [3]:
#pip install happytransformer

In [4]:
#pip install nltk

In [5]:
#!python -m spacy download en_core_web_sm

In [6]:
#pip install wordcloud

In [7]:
#nltk.download('wordnet')

https://www.kaggle.com/datasets/nidhiy07/student-depression-text

https://www.kaggle.com/code/nidhiy07/forecast-depression-anxiety-text

1) Este conjunto de datos contiene información en formato Excel que comprende unos 7489 datos de redes sociales, comentarios de Facebook, etc.
2) Todas las personas seleccionadas para la anotación de datos dominan muy bien el inglés y son estudiantes, con edades comprendidas entre los 15 y los 17 años.
3)Hay 5 columnas en este conjunto de datos. Texto, etiquetas, edad, categoría de edad y sexo.
4) En las columnas de texto hay texto normal y de ansiedad/depresión, y la columna de etiquetas indica si el texto correspondiente denota ansiedad o depresión.
1 ANSIEDAD
0 NEUTRO

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim #Pytorch
from datasets import load_dataset

In [2]:
RANDOM_SEED = 42
BATCH_SIZE = 16
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Cargar datos

In [3]:
dir_pandas = 'C:/Users/anabe/Documents/PRUEBAS/GPT/corpus/{}'
df=pd.read_excel(dir_pandas.format('depresion.xlsx'))
#Solo para ver los primeros
df.head(10)

,text,label,Age,Gender,Age Category
0,let Gather For Party,0.0,16.0,Male,Teen Age
1,I hate being alive when I feel so dead inside.,1.0,17.0,Female,Teen Age
2,Exam are Comimg I m already Feeling Very Dep...,1.0,16.0,Male,Teen Age
3,Nice picture,0.0,16.0,Male,Teen Age
4,"Feeling worried, even though you actually have...",1.0,16.0,Male,Teen Age
5,"There are people who died, their friends didn'...",0.0,16.0,Male,Teen Age
6,keep it up hahaha,0.0,16.0,Male,Teen Age
7,The match cycle I had.Buy-brought-out-lost.,0.0,16.0,Male,Teen Age
8,"nctb! jewel case dream, which doesn't have a p...",0.0,16.0,Male,Teen Age
9,"college but it's like not going to college, bu...",0.0,16.0,Male,Teen Age


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7489 entries, 0 to 7488
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   text          7476 non-null   object 
 1   label         7486 non-null   float64
 2   Age           7486 non-null   float64
 3   Gender        7486 non-null   object 
 4   Age Category  7486 non-null   object 
dtypes: float64(2), object(3)
memory usage: 292.7+ KB


In [5]:
# Checking null values
df.isnull().sum()

text            13
label            3
Age              3
Gender           3
Age Category     3
dtype: int64

In [6]:
df.describe()

,label,Age
count,7486.000000,7486.000000
mean,0.163906,15.379108
std,0.370215,1.058104
min,0.000000,13.000000
25%,0.000000,14.000000
50%,0.000000,15.000000
75%,0.000000,16.000000
max,1.000000,17.000000


In [7]:
dup_df = df[df.duplicated()].shape[0]
print(f"There are {dup_df} duplicate entries among {df.shape[0]} entries in  dataset.")

There are 39 duplicate entries among 7489 entries in  dataset.


In [8]:
df = df.sample(frac = 1)

In [9]:
df['label'].value_counts().to_frame().T.style.set_properties(**{"background-color": "#F3FFFF","color":"black","border": "1.5px  solid black"})

label,0.000000,1.000000
count,6259,1227


In [10]:
df.head()

,text,label,Age,Gender,Age Category
2827,"It's okay, I also have no intention to be angr...",0.0,14.0,Male,Young Age
2858,COMO Q JYP RIPEO,0.0,14.0,Male,Young Age
2133,Pharmaceutical stocks are in ruins right now. ...,0.0,17.0,Male,Teen Age
4357,â€œSuicide it too hard to commitâ€Watch me. G...,1.0,16.0,Female,Teen Age
2908,It's not really nice to be ignored ðŸ˜©,0.0,14.0,Male,Young Age


In [11]:
df['label'].value_counts()

label
0.0    6259
1.0    1227
Name: count, dtype: int64

In [12]:
not_used_cols = ['Age','Gender','Age Category']
df1 = df.drop(not_used_cols,axis=1)
df1.sample(3)

,text,label
5839,qn es yuri,0.0
4953,"In the previous project, my uname is different...",0.0
3581,"I'm so nervous I swear, the views are getting ...",1.0


In [13]:
df1['label'].unique()

array([ 0.,  1., nan])

In [14]:
df1 = df1.dropna(axis=0)
print("DataFrame after removing rows with NaN value in any column:")
print(df1)

DataFrame after removing rows with NaN value in any column:
                                                   text  label
2827  It's okay, I also have no intention to be angr...    0.0
2858                                   COMO Q JYP RIPEO    0.0
2133  Pharmaceutical stocks are in ruins right now. ...    0.0
4357  â€œSuicide it too hard to commitâ€Watch me. G...    1.0
2908            It's not really nice to be ignored ðŸ˜©    0.0
...                                                 ...    ...
5191  Looking for mutualan with fellow nctzens, who ...    0.0
5226                                 Ami ajke onk happy    0.0
5390                              como que jyp mur10 ee    0.0
860      Arya, if you play futsal, you don't use a goal    0.0
7270             I feel excited . Gosh, a little tremor    0.0

[7476 rows x 2 columns]


In [15]:
df1.shape

(7476, 2)

In [16]:
df1['label'].unique()

array([0., 1.])

In [17]:
df.dtypes

text             object
label           float64
Age             float64
Gender           object
Age Category     object
dtype: object

In [18]:
df1['label'] = df1['label'].astype('int64')
df1.dtypes

text     object
label     int64
dtype: object

In [19]:
df1.shape

(7476, 2)

In [20]:
df1

,text,label
2827,"It's okay, I also have no intention to be angr...",0
2858,COMO Q JYP RIPEO,0
2133,Pharmaceutical stocks are in ruins right now. ...,0
4357,â€œSuicide it too hard to commitâ€Watch me. G...,1
2908,It's not really nice to be ignored ðŸ˜©,0
...,...,...
5191,"Looking for mutualan with fellow nctzens, who ...",0
5226,Ami ajke onk happy,0
5390,como que jyp mur10 ee,0
860,"Arya, if you play futsal, you don't use a goal",0


In [21]:
df1.to_csv('C:/Users/anabe/Documents/PRUEBAS/GPT/corpus/df.csv',header=True, index=False)